In [3]:
import numpy as np
import galpy
import galpy.df
import galpy.potential
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as con


In [4]:
G = con.G.to(u.pc**3 / u.solMass / u.yr**2)
G

<Quantity 4.49850215e-15 pc3 / (solMass yr2)>

# Create Distributions of Points

In [99]:
def generate_points_spherical(N, M, DF_name):
    """
    Use galpy to generate spherical distributions 
    of particles we can feed into the LAMAR code. 
    
    For reference 
    M15:
    Mass = 450,000 solar masses
    N    = 10^5 stars
    rt   = 60 pc
    
    Args:
    N - int - Number of Particles being Used
    M - float - total mass of the globular cluster in solar masses
    DF_name - string - name of the distribution function being used
    
    Returns:
    x,y,z - nd.array - positions in units of pc
    vx,vy,vz - nd.array - velocities in units of pc/yr
    m - nd.array - masses in units of solar masses
    
    """
    
    if DF_name == 'king':
        
        #values for M15 Globular Cluster for Reference
        # rt_M15 = 0.061*u.kpc ; M_M15  = 4.5e5*u.solMass
        
        rt = 0.061 * u.kpc
        DF = galpy.df.kingdf(M=M * u.solMass, rt=rt, W0=3., ro=8*u.kpc, vo=220*(u.km/u.s))
        
    elif DF_name == 'plummer':
        
        amp = G * (M * u.solMass)
        pot=galpy.potential.PlummerPotential(amp=amp, b=0.8, ro=8*u.kpc, vo=220*(u.km/u.s))
        DF = galpy.df.isotropicPlummerdf(pot=pot, ro=8*u.kpc, vo=220*(u.km/u.s))
        
    elif DF_name == 'nfw':
        amp = M * u.solMass
        a   = 15 * u.kpc
        NFW_pot = galpy.potential.NFWPotential(amp=amp, a=a, ro=8*u.kpc, vo=220*(u.km/u.s))
        DF = galpy.df.isotropicNFWdf(pot=NFW_pot, widrow=True, rmax=300 * u.kpc ,ro=8*u.kpc, vo=220*(u.km/u.s))

    
    else:
        raise AssertionError("Not Implemented yet...")
        
    
    R, vR, vT, z, vz, phi = DF.sample(n=int(N),  return_orbit=False)
    x = R * np.cos(phi)
    y = R * np.sin(phi)
    z = z

    vx = vR * np.cos(phi) - vT * np.sin(phi)
    vy = vR * np.sin(phi) + vT * np.cos(phi)
    vz = vz
    
    x = x * 8 * u.kpc
    y = y * 8 * u.kpc
    z = z * 8 * u.kpc
    
    vx = vx * 220 * u.km/u.s
    vy = vy * 220 * u.km/u.s
    vz = vz * 220 * u.km/u.s
    
    m = np.ones_like(x.value) * (M / N) * u.solMass #evenly divide mass across all stars
    
    
    return [x.to(u.pc), y.to(u.pc), z.to(u.pc), vx.to(u.pc/u.yr), vy.to(u.pc/u.yr), vz.to(u.pc/u.yr), m.to(u.solMass)]
    
    
def generate_points_disk(N, M, DF_name, add_BH=False):
    """
    Use galpy to generate razor-thin disk distributions 
    of particles we can feed into the LAMAR code. 
    
    Args:
    N - int - Number of Particles being Used
    DF_name - string - name of the distribution function being used
    
    Returns:
    x,y,z - nd.array - positions in units of pc
    vx,vy,vz - nd.array - velocities in units of pc/yr
    m - nd.array - masses in units of solar masses
    
    """
    
    if DF_name == 'dehnen':
        DF = galpy.df.dehnendf(beta=0.)
       
    elif DF_name == 'shu':
        DF = galpy.df.shudf(beta=0.)
    
    elif DF_name == 'schwarzschild':
        DF = galpy.df.schwarzschilddf(beta=0.)
        
    else:
        raise AssertionError("Not Implemented yet...")
        
    samples = DF.sample(n=int(N), returnOrbit=True, nphi=1, correct=True, ro=8*u.kpc, vo=220*(u.km/u.s))
    
    x = np.array([e.x() for e in samples])
    y = np.array([e.y() for e in samples])
    z = np.zeros_like(x)

    vx = np.array([e.vx() for e in samples])
    vy = np.array([e.vy() for e in samples])
    vz = np.zeros_like(vx)
    
    m = np.ones_like(x) * (M / N) * u.solMass #evenly divide mass across all stars
    
    if add_BH:
        #set a supermassive BH at the center of the disk with a large mass
        x[0] = 0 ; y[0] = 0 ; z[0] = 0
        vx[0] = 0 ; vy[0] = 0 ; vz[0] = 0
        m[0] = 1e11 * u.solMass
        
    
    #add units to position
    x = x * 8 * u.kpc ; y = y * 8 * u.kpc ;z = z * 8 * u.kpc
    
    #add units to velocity
    vx = vx * 220 * u.km/u.s ; vy = vy * 220 * u.km/u.s ; vz = vz * 220 * u.km/u.s
    
    return [x.to(u.pc), y.to(u.pc), z.to(u.pc), vx.to(u.pc/u.yr), vy.to(u.pc/u.yr), vz.to(u.pc/u.yr), m.to(u.solMass)]
    

def generate_points_ring(N, M):
    """
    Generate a distribution of points in an annulus ring
    of width 2 kpc with a central massive particle. This
    distribution is used to test to ensure the gravity 
    is working.
    """
    
    Rmin = 8  #kpc
    Rmax = 10 #kpc
    
    R   = np.random.uniform(Rmin, Rmax, N)
    phi = np.random.uniform(0, 2*np.pi, N)
    z   = np.zeros(N)
    
    x = R * np.cos(phi) * u.kpc
    y = R * np.sin(phi) * u.kpc
    z = z * u.kpc
    
    vx = np.zeros(N) * (u.km/u.s)
    vy = np.zeros(N) * (u.km/u.s)
    vz = np.zeros(N) * (u.km/u.s)
    
    m = np.ones(N) * (M / N) * u.solMass #solmass
    
    
    x[0], y[0], z[0] = 0.*u.kpc, 0.*u.kpc, 0.*u.kpc
    vx[0], vy[0], vz[0] = 0.* (u.km/u.s), 0.* (u.km/u.s), 0.* (u.km/u.s)
    m[0] = 1e10 * u.solMass
    
    return [x.to(u.pc), y.to(u.pc), z.to(u.pc), vx.to(u.pc/u.yr), vy.to(u.pc/u.yr), vz.to(u.pc/u.yr), m.to(u.solMass)]
    

def generate_points_galaxy(N):
    """
    Generate a mock distribution of particles that represents
    the Milky Way galaxy. We have 3 components, the DM halo, the 
    disk, and the central BH. All values are approximate.
    """
    
    N_halo = N // 2
    N_disk = N // 2
    
    M_halo  = 1e12
    M_disk  = 1e10
    M_BH    = 1e6
    
    print("Making the Disk")
    points_disk = generate_points_disk(N_disk, M_disk, 'dehnen', add_BH=False)
    print("Making the Halo")
    points_halo = generate_points_spherical(N_halo, M_halo, DF_name='nfw')
    
    
    points = np.concatenate((np.array(points_disk).T, np.array(points_halo).T), axis=0)
    points[0] = [0, 0, 0, 0, 0, 0, M_BH] 
    
    return points

def generate_points_spiral_disk(N, M, Rmin=1, Rmax=8, m=1, M_BH=None, disk_type='keplerian'):
    """
    Generate the points for a razor-thin keplerian disk with spiral arms. 
    The arms are generated using cos(mφ) angular profile. We have 25% of the 
    mass is in the spiral arms, 75% in the rest of the disk.
    
    Args:
    N - int - number of points.
    M - float - mass of the disk
    Rmin - float minimum radius of stars in the disk (in kpc)
    Rmax - float maximum radius of stars in the disk (in kpc)
    m - int - number of spiral arms 
    M_BH - float/none - add a central black hole if not None
    disk_type - str - type of disk. Default is keplerian, other option is 'flat'
    
    Returns:
    points array
    """

    M = M * u.solMass
    
    N_spiral_1 = N_spiral_2 = N // 8
    N_disk = int((3/4) * N)
    
    M_spiral_1 = M_spiral_2 = M / 8
    M_disk = (3/4) * M
    

    # FIRST SPIRAL PATTERN
    r_spiral_1 = np.linspace(Rmin, Rmax, N // 8) * u.kpc
    phi_spiral_1 = np.linspace(0, 2*np.pi, N // 8)
    m_spiral_1 = np.ones(N_spiral_1) * (M_spiral_1 / N_spiral_1)

    x_spiral_1 = r_spiral_1 * np.cos(m*phi_spiral_1)
    y_spiral_1 = r_spiral_1 * np.sin(m*phi_spiral_1)
    
    if disk_type == 'flat':
        vT = 200 * (u.km/u.s)
        vR = 0
    elif disk_type == 'keplerian':
        vT = np.sqrt(G * m_spiral_1 / r_spiral_1)
        vR = 0
    else:
        raise AssertionError("not implemented yet...")

    vx_spiral_1 = vR * np.cos(phi_spiral_1) - vT * np.sin(phi_spiral_1)
    vy_spiral_1 = vR * np.sin(phi_spiral_1) + vT * np.cos(phi_spiral_1)

    # SECOND SPIRAL PATTERN
    r_spiral_2 = np.linspace(Rmin, Rmax, N // 8) * u.kpc
    phi_spiral_2 = np.linspace(0, 2*np.pi, N // 8) + np.pi
    m_spiral_2 = np.ones(N_spiral_2) * (M_spiral_2 / N_spiral_2)

    x_spiral_2 = r_spiral_2 * np.cos(m*phi_spiral_2)
    y_spiral_2 = r_spiral_2 * np.sin(m*phi_spiral_2)

    if disk_type == 'flat':
        vT = 200 * (u.km/u.s)
        vR = 0
    elif disk_type == 'keplerian':
        vT = np.sqrt(G * m_spiral_2 / r_spiral_2)
        vR = 0
    else:
        raise AssertionError("not implemented yet...")

    vx_spiral_2 = vR * np.cos(phi_spiral_2) - vT * np.sin(phi_spiral_2)
    vy_spiral_2 = vR * np.sin(phi_spiral_2) + vT * np.cos(phi_spiral_2)

    # THE REST IS UNIFORM DISK
    r = np.random.uniform(low=Rmin, high=Rmax, size=int((3/4) * N)) * u.kpc
    phi = np.random.uniform(low=0, high=2 * np.pi, size=int((3/4) * N))
    m = np.ones(N_disk) * (M_disk / N_disk)

    x = r * np.cos(phi)
    y = r * np.sin(phi)

    if disk_type == 'flat':
        vT = 200 * (u.km/u.s)
        vR = 0
    elif disk_type == 'keplerian':
        vT = np.sqrt(G * m / r)
        vR = 0
    else:
        raise AssertionError("not implemented yet...")

    vx = vR * np.cos(phi) - vT * np.sin(phi)
    vy = vR * np.sin(phi) + vT * np.cos(phi)

    #combine all stars in the disk with the spiral patterns
    x = np.concatenate((x_spiral_1, x_spiral_2, x), axis=0)
    y = np.concatenate((y_spiral_1, y_spiral_2, y), axis=0)
    z = np.zeros_like(x)

    vx = np.concatenate((vx_spiral_1, vx_spiral_2, vx), axis=0)
    vy = np.concatenate((vy_spiral_1, vy_spiral_2, vy), axis=0)
    vz = np.zeros_like(vx)
    
    m = np.concatenate((m_spiral_1, m_spiral_2, m), axis=0)
    
    if M_BH is not None:
        x[0], y[0], z[0] = 0. * u.kpc,0. * u.kpc,0. * u.kpc
        vx[0], vy[0], vz[0] = 0. * u.km/u.s,0. * u.km/u.s,0. * u.km/u.s
        m[0] = M_BH * u.solMass


    points = [x.to(u.pc),y.to(u.pc),z.to(u.pc), vx.to(u.pc/u.yr),vy.to(u.pc/u.yr),vz.to(u.pc/u.yr), m.to(u.solMass)]

    return points


def generate_points_uniform_disk(N, M, Rmin=1, Rmax=8, m=1, M_BH=None, disk_type='keplerian'):
    """
    Generate the points for a razor-thin keplerian disk with spiral arms. 
    The arms are generated using cos(mφ) angular profile. We have 25% of the 
    mass is in the spiral arms, 75% in the rest of the disk.
    
    Args:
    N - int - number of points.
    M - float - mass of the disk
    Rmin - float minimum radius of stars in the disk (in kpc)
    Rmax - float maximum radius of stars in the disk (in kpc)
    m - int - number of spiral arms 
    M_BH - float/none - add a central black hole if not None
    disk_type - str - type of disk. Default is keplerian, other option is 'flat'
    
    Returns:
    points array
    """

    M_disk = M * u.solMass
    N_disk = N
    
    # THE REST IS UNIFORM DISK
    r = np.random.uniform(low=Rmin, high=Rmax, size=N_disk) * u.kpc
    phi = np.random.uniform(low=0, high=2 * np.pi, size=N_disk)
    m = np.ones(N_disk) * (M_disk / N_disk)

    x = r * np.cos(phi)
    y = r * np.sin(phi)
    z = np.zeros_like(x)

    if disk_type == 'flat':
        vT = 200 * (u.km/u.s)
        vR = 0
    elif disk_type == 'keplerian':
        vT = np.sqrt(G * m / r).to(u.km/u.s)
        vR = 0
    else:
        raise AssertionError("not implemented yet...")

    vx = vR * np.cos(phi) - vT * np.sin(phi)
    vy = vR * np.sin(phi) + vT * np.cos(phi)
    vz = np.zeros_like(vx)
    
    if M_BH is not None:
        x[0], y[0], z[0] = 0. * u.kpc,0. * u.kpc,0. * u.kpc
        vx[0], vy[0], vz[0] = 0. * u.km/u.s,0. * u.km/u.s,0. * u.km/u.s
        m[0] = M_BH * u.solMass


    points = [x.to(u.pc),y.to(u.pc),z.to(u.pc), vx.to(u.pc/u.yr),vy.to(u.pc/u.yr),vz.to(u.pc/u.yr), m.to(u.solMass)]

    return points

# Generate Initialization Files

In [138]:
def generate_init_file(N_particles, t_max, dt, skip_Nsteps, simulation_type):
    """
    generate the initial conditions file.
    """
    
    if simulation_type == 'merger':
        
        print("Generating Disk 1 ...")
        points1 = generate_points_uniform_disk(N=N_particles//2, M=1e6, Rmin=1, Rmax=4, m=1, M_BH=1e12, disk_type='flat')
        points2 = points1.copy()
        
        #shift the disk
        x1,y1,z1,vx1,vy1,vz1,m1 = points1
        
        x1 = ((x1.to(u.kpc).value - 6) * u.kpc).to(u.pc)
        y1 = ((y1.to(u.kpc).value - 6) * u.kpc).to(u.pc)
        
        # force disk to move towards the other disk
        vx1 = ((vx1.to(u.kpc/u.yr).value + 12e-8) * (u.kpc/u.yr)).to(u.pc/u.yr)
        # vy1 = ((vx1.to(u.kpc/u.yr).value + 1e-8) * (u.kpc/u.yr)).to(u.pc/u.yr)
        
        points1 = [x1,y1,z1,vx1,vy1,vz1,m1]
        
        #transpose
        points1_T = np.array(points1).T

        print("Generating Disk 2 ...")
        # points2 = generate_points_disk(N=N_particles // 2, M=1e5, DF_name = 'dehnen', add_BH=True)
        
        #shift the disk
        x2,y2,z2,vx2,vy2,vz2,m2 = points2
        x2 = ((x2.to(u.kpc).value + 6) * u.kpc).to(u.pc)
        y2 = ((y2.to(u.kpc).value + 6) * u.kpc).to(u.pc)
        
        #force disk to move towards the other disk
        vx2 = ((vx2.to(u.kpc/u.yr).value - 12e-8) * (u.kpc/u.yr)).to(u.pc/u.yr)
        # vy2 = ((vx2.to(u.kpc/u.yr).value - 1e-8) * (u.kpc/u.yr)).to(u.pc/u.yr)
        
        points2 = [x2,y2,z2,vx2,vy2,vz2,m2]
        
        #transpose
        points2_T = np.array(points2).T
        
        #combine both disks' points into one array
        points_T = np.concatenate((points1_T, points2_T), axis=0)
     
        
    elif simulation_type == 'globular_cluster':
        
        points = generate_points_spherical(N=N_particles, M=4.5e5, DF_name='king')        
        points_T = np.array(points).T
        
        
    elif simulation_type == 'disk':

        points = generate_points_disk(N=N_particles, M=1e5, DF_name = 'dehnen', add_BH=True)
        points_T = np.array(points).T
        
    elif simulation_type == 'uniform_disk':
        points = generate_points_uniform_disk(N=N_particles, M=1e6, Rmin=1, Rmax=4, m=1, M_BH=1e10, disk_type='flat')
        points_T = np.array(points).T
        
    elif simulation_type == 'spiral_disk':
        points = generate_points_spiral_disk(N=N_particles, M=1e7, Rmin=1, Rmax=8, m=1, M_BH=5e10, disk_type='flat')
        points_T = np.array(points).T
        
        
    elif simulation_type == 'ring':
        points = generate_points_ring(N=N_particles, M=1e5)
        points_T = np.array(points).T
        
    elif simulation_type == 'galaxy':
        points = generate_points_galaxy(N = N_particles)
        points_T = np.array(points).T
        
        
    else:
        raise AssertionError("This hasnt been implemented yet...")
        
        
    #save data
    savename = simulation_type + '_init_file.txt'
    np.savetxt('../src/' + savename, points_T, 
               fmt='%.18f', delimiter=" ", 
               header=f'{N_particles} {t_max} {dt} {skip_Nsteps}', 
               comments='')
    
    return None

# Save Initialization Files

In [6]:
generate_init_file(N_particles=int(1e4), 
                   t_max=1e6, 
                   dt=10, 
                   skip_Nsteps=1000, 
                   simulation_type='globular_cluster')

In [139]:
generate_init_file(N_particles=int(4e3), 
                   t_max=1e8, 
                   dt=1e3, 
                   skip_Nsteps=1000, 
                   simulation_type='merger')

Generating Disk 1 ...
Generating Disk 2 ...


In [28]:
generate_init_file(N_particles=int(1e3), 
                   t_max=1e8, 
                   dt=1000, 
                   skip_Nsteps=1000, 
                   simulation_type='disk')

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 9.769898e-05
  warnings.warn(

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 7.423495e-05
  warnings.warn(

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 1.296897e-05
  warnings.warn(

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 9.852781e-06
  warnings.warn(

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 8.695383e-05
  warnings.warn(

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 8.235027e-05
  warnings

In [67]:
generate_init_file(N_particles=int(4e3), 
                   t_max=1e9, 
                   dt=1e3, 
                   skip_Nsteps=1000, 
                   simulation_type='spiral_disk')

In [126]:
generate_init_file(N_particles=int(4e3), 
                   t_max=1e9, 
                   dt=1e4, 
                   skip_Nsteps=100, 
                   simulation_type='uniform_disk')

In [42]:
generate_init_file(N_particles=int(1e3), 
                   t_max=5e8, 
                   dt=1000, 
                   skip_Nsteps=1000, 
                   simulation_type='ring')

In [93]:
generate_init_file(N_particles=int(1e3), 
                   t_max=1e8, 
                   dt=1000, 
                   skip_Nsteps=1000, 
                   simulation_type='galaxy')

Making the Disk
/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 4.415158e-05
  warnings.warn(

/home/js7501/.local/lib/python3.11/site-packages/galpy/util/quadpack.py:136: AccuracyWarning: maxiter (50) exceeded. Latest difference = 6.856831e-06
  warnings.warn(

Making the Halo
